# Assignment 2 Questions #
#### CSCI 4831 - Sabermetrics ####
#### Instructor: Hoenigman ####
#### Due: Wednesday, Feb 6 by 3pm ####

Name: Michael Dresser

**Using the Statcast data, answer the following questions in a Jupyter Notebook. These questions all use data from the 2017 season. I recommend that you download all of the 2017 data first and store it as a csv, and then read from the csv.** 

**Submit your notebook to the Assignment 2 link on Canvas. Include all csv files that you generate with your submission.**

1. (10 pts) Which Rockies player had the most plate appearances with bases loaded in 2017?

In [46]:
import pandas as pd
april = pd.read_csv("data/dataApril2017.csv")#, index_col=1)
may = pd.read_csv("data/dataMay2017.csv")#, index_col=1)
june = pd.read_csv("data/dataJune2017.csv")#, index_col=1)
july = pd.read_csv("data/dataJuly2017.csv")#, index_col=1)
august = pd.read_csv("data/dataAug2017.csv")#, index_col=1)
september = pd.read_csv("data/dataSept2017.csv")#, index_col=1)
october = pd.read_csv("data/dataOct2017.csv")#, index_col=1)

months = [april, may, june, july, august, september, october]

for i in range(len(months)):
    df = months[i]
    df.drop("Unnamed: 0", axis=1, inplace=True)
    df.drop("index", axis=1, inplace=True)
    #df.drop("Index")
    df = df.reindex(sorted(df.columns), axis=1)
    months[i] = df

df2017 = pd.concat(months, sort=True)

In [49]:
s = 0
for df in months:
    s += len(df)
    
print(s)
print(len(df2017))

df2017 = df2017.reset_index(drop=True)
print(df2017.columns)

712505
712505
Index(['at_bat_number', 'away_score', 'away_team', 'ax', 'ay', 'az',
       'babip_value', 'balls', 'bat_score', 'batter', 'bb_type',
       'break_angle_deprecated', 'break_length_deprecated', 'des',
       'description', 'effective_speed', 'error',
       'estimated_ba_using_speedangle', 'estimated_woba_using_speedangle',
       'events', 'fielder_2', 'fielder_2.1', 'fielder_3', 'fielder_4',
       'fielder_5', 'fielder_6', 'fielder_7', 'fielder_8', 'fielder_9',
       'fld_score', 'game_date', 'game_pk', 'game_type', 'game_year', 'hc_x',
       'hc_y', 'hit_distance_sc', 'hit_location', 'home_score', 'home_team',
       'if_fielding_alignment', 'inning', 'inning_topbot', 'iso_value',
       'launch_angle', 'launch_speed', 'launch_speed_angle',
       'of_fielding_alignment', 'on_1b', 'on_2b', 'on_3b', 'outs_when_up',
       'p_throws', 'pfx_x', 'pfx_z', 'pitch_name', 'pitch_number',
       'pitch_type', 'pitcher', 'pitcher.1', 'plate_x', 'plate_z',
       'player_name'

In [48]:
df_rockies_home = df2017.loc[df2017["home_team"] == "COL"]
df_rockies_away = df2017.loc[df2017["away_team"] == "COL"]

df_rockies_home_batting = df_rockies_home.loc[df2017["inning_topbot"] == "Bot"]
df_rockies_away_batting = df_rockies_away.loc[df2017["inning_topbot"] == "Top"]
df_rockies_batting = pd.concat([df_rockies_home_batting, df_rockies_away_batting])

In [50]:
df_rockies_batting_loaded = df_rockies_batting.loc[(df_rockies_batting["on_1b"].notna()) &
                                                   (df_rockies_batting["on_2b"].notna()) &
                                                   (df_rockies_batting["on_3b"].notna())] 

In [56]:
df_rockies_batting_loaded.head(10)

,at_bat_number,away_score,away_team,ax,ay,az,babip_value,balls,bat_score,batter,...,tfs_deprecated,tfs_zulu_deprecated,type,umpire,vx0,vy0,vz0,woba_denom,woba_value,zone
16218,45.0,5.0,WSH,-13.4047,23.1589,-20.9856,0.0,2.0,1.0,571448.0,...,NaN,NaN,X,NaN,9.8604,-135.2063,-2.3329,1.0,0.0,2.0
16219,45.0,5.0,WSH,2.6727,21.6063,-22.9692,NaN,1.0,1.0,571448.0,...,NaN,NaN,B,NaN,9.6788,-126.5445,-3.4772,NaN,NaN,12.0
16220,45.0,5.0,WSH,0.7319,20.6756,-25.7949,NaN,0.0,1.0,571448.0,...,NaN,NaN,B,NaN,8.6497,-126.1199,-5.7306,NaN,NaN,14.0
16221,44.0,5.0,WSH,1.7905,22.1229,-25.7815,0.0,3.0,0.0,518934.0,...,NaN,NaN,B,NaN,9.8861,-125.4170,-3.7960,1.0,0.7,14.0
16222,44.0,5.0,WSH,-14.0243,25.3771,-18.9556,NaN,3.0,0.0,518934.0,...,NaN,NaN,S,NaN,10.1563,-135.3208,-2.8729,NaN,NaN,2.0
16223,44.0,5.0,WSH,-13.3943,26.1137,-17.6384,NaN,2.0,0.0,518934.0,...,NaN,NaN,B,NaN,11.9591,-135.6810,-4.7351,NaN,NaN,12.0
16224,44.0,5.0,WSH,-5.5731,25.1076,-16.8614,NaN,1.0,0.0,518934.0,...,NaN,NaN,B,NaN,5.5470,-132.2126,2.8563,NaN,NaN,11.0
16225,44.0,5.0,WSH,-16.0289,25.3183,-17.5920,NaN,1.0,0.0,518934.0,...,NaN,NaN,S,NaN,10.7931,-136.2630,-6.4251,NaN,NaN,9.0
16226,44.0,5.0,WSH,0.1116,22.7544,-27.9620,NaN,1.0,0.0,518934.0,...,NaN,NaN,S,NaN,8.8887,-124.4359,-2.3802,NaN,NaN,6.0
16227,44.0,5.0,WSH,-9.5421,23.5376,-18.9859,NaN,0.0,0.0,518934.0,...,NaN,NaN,B,NaN,8.0745,-132.6484,-1.3931,NaN,NaN,11.0


2. (10 pts) Which Rockies pitcher threw the most first-pitch strikes in 2017?

3. (10 pts) How many times did a Rockies hitter cause a lead change in a game in the seventh inning or later in 2017? i.e. The Rockies were losing and after the player's plate appearance, the Rockies were winning. This doesn't mean that they ultimately won the game.

4. (70 pts) So much is made about the Rockies pitching staff and how it's difficult to pitch at Coors Field because of the altitude. In the thin air, breaking balls don't break as much as they do at sea level, which makes them easier to hit. We might wonder if pitchers change their approach to pitching when they come to Coors Field, or do Rockies pitchers pitch differently at Coors than they do at other ballparks. 

For this question, you need to evaluate the starting pitching staff for the 2017 Rockies to determine if there is a difference in how they approach pitching at home and away. 

Select the five pitchers who started the most games for the Rockies in 2017.

Select the data for each game that they started at home, and games they started against the other NL West teams (San Diego, Los Angeles, San Francisco, and Arizona).

Compare pitch selection for each pitcher at home and away. You can use the mean number of pitch types, or a more advanced statistical measure if you prefer.

Compare the movement of their pitches at home and away. Again, you can use the mean.

Choose a situation, such as first pitch of the inning, runners on base and no on out, etc, and compare their pitch selection in that situation at home and away. 

Generate the code to answer these questions and then write about 2 pages explaining your results. You explanation should include 
* An introduction to the problem
* The data that you used
* Your process, including a summary of what your code does. (You don't need to show your code here.)
* Your results. It will be good to include some tables or graphs here.
* Limitations of your analysis, other variables not considered that could affect your findings.